In [1]:
# Import libraries
import numpy as np
import pandas as pd
import sys
import os
import fitz
from unidecode import unidecode 
import re
import datetime
from spacy import displacy
import spacy
import unicodedata

In [2]:
cv_folder = '../data/cv'

In [176]:
cv_file_name = 'CV Melvin Leal.pdf'#'Resume-Franklin-Herrera (1).pdf'
cv_file_name = '202206 Javier Andújar CV.pdf'
#cv_file_name = 'José Luis Gutiérrez Mendoza.docx.pdf'


In [177]:
DIGITIZED_FILE = os.path.join(cv_folder,cv_file_name)

In [226]:
with fitz.open(DIGITIZED_FILE) as doc:
    block_dict = {}
    page_num = 1
    line_num_test = 1
    
    for page in doc: # Iterate all pages in the document
        file_dict = page.get_text('dict') # Get the page dictionary 
        block = file_dict['blocks'] # Get the block information
        
        for a in block:   
            if a["type"] == 0:
                for line in a['lines']:
                    for span in line['spans']:
                        span["page_num"] = page_num
                        span["line_num"] =  line_num_test

                line_num_test += 1

            else :
                a["page_num"] = page_num

        block_dict[page_num] = block # Store in block dictionary

        page_num += 1 # Increase the page value by 1

In [180]:
rows = []

for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    color = span['color']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    num_page = span["page_num"]
                    line_num = span['line_num']
                    block_num = block['number']
                    is_upper = False
                    is_bold = False 
                    if "bold" in span_font.lower():
                        is_bold = True 
                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True
                    if text.replace(" ","") !=  "":
                        rows.append((xmin, ymin, xmax, ymax, line_num, block_num, num_page, text, is_upper, is_bold, span_font, font_size, color))
                        

span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'line_num', 'block_num', "page_num", 'text', 'is_upper','is_bold','span_font', 'font_size', 'color'])

In [182]:
span_scores = []
span_num_occur = {}
special = '[(_:/,#%\=@)]'

for index, span_row in span_df.iterrows():
    score = round(span_row.font_size)
    text = span_row.text

    if not re.search(special, text):
        if span_row.is_bold:
            score +=1 

        if span_row.is_upper:
            score +=1
    span_scores.append(score)
values, counts = np.unique(span_scores, return_counts=True)

In [183]:
values, counts = np.unique(span_scores, return_counts=True)
style_dict = {}
for value, count in zip(values, counts):
    style_dict[value] = count
sorted(style_dict.items(), key=lambda x: x[1])

[(17, 2), (8, 3), (18, 3), (12, 7), (13, 13), (10, 22), (11, 108)]

In [184]:
p_size = max(style_dict, key=style_dict.get)
idx = 0
tag = {}

for size in sorted(values, reverse = True):
    idx += 1
    if size == p_size:
        idx = 0
        tag[size] = 'p'
    if size > p_size:
        tag[size] = 'h{0}'.format(idx)
    if size < p_size:
        tag[size] = 's{0}'.format(idx)

In [185]:
span_tags = [tag[score] for score in span_scores]

span_df['tag'] = span_tags

## Data Handling

### Columns Pre bucketing 

As can be seen in the dataframe, there is text from the same line separated by rows. This for/while loop joins them together with its correct bbox values per variable.

In [198]:
## Merge text from same line
deletion = []

for index, line in enumerate(span_df.line_num):
  
    try :
        contador = 1
        while span_df.loc[index, 'line_num'] == span_df.loc[(index + contador), 'line_num']: 
            span_df.loc[index, 'text'] = span_df.loc[index,'text'] + " " + span_df.loc[index+contador, 'text']
            deletion.append(index+contador)
            contador += 1

        if contador > 1:
            span_df.loc[index, 'xmin'] = [min(span_df.loc[index:index+contador - 1,'xmin'])]
            span_df.loc[index, 'ymin'] = [min(span_df.loc[index:index+contador - 1,'ymin'])]
            span_df.loc[index, 'xmax'] = [max(span_df.loc[index:index+contador - 1,'xmax'])]
            span_df.loc[index, 'ymax'] = [max(span_df.loc[index:index+contador - 1,'ymax'])]

    except KeyError as error :
        continue

In [199]:
span_df.drop(deletion, axis = 0, inplace= True)
span_df.reset_index(drop = True, inplace = True)

In [225]:
## Identify columns patterns in first page
## Assume first page columns pattern is replicated over pages
page_one = span_df[span_df['page_num']==1]

main_col_x_min = page_one['xmin'].value_counts().index[0]
main_col_x_max = page_one[page_one['xmin']==main_col_x_min]['xmax'].min()
main_col_y_min = page_one[page_one['xmin']==main_col_x_min]['ymin'].min()
page_one['flg_other_column']=((page_one['xmax']<main_col_x_min )| (page_one['xmin']>main_col_x_max ))*1

span_df['column']='main'
if page_one['flg_other_column'].sum()>=5:
    span_df.loc[((span_df['xmax']<main_col_x_min )| (span_df['xmin']>main_col_x_max )),'column']='secondary'
    print('2 columns in the document')
display(span_df.sample(3))    

2 columns in the document


/var/folders/v2/qknnv4n91jg_n5dynkby31nh0000gp/T/ipykernel_46323/2192374198.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page_one['flg_other_column']=((page_one['xmax']<main_col_x_min )| (page_one['xmin']>main_col_x_max ))*1


,xmin,ymin,xmax,ymax,line_num,block_num,page_num,text,is_upper,is_bold,span_font,font_size,color,tag,column
87,461.899994,309.429962,487.116638,317.469971,131,14,4,Spanish English,False,False,Calibri,8.04,7105644,s2,secondary
30,30.600000,59.429962,136.177902,75.389961,38,37,1,Javier Gonzalez,False,True,"Calibri,Bold",15.96,16777215,h2,secondary
8,217.250000,196.429962,381.281708,206.389969,12,11,1,Buenos Aires City -- Nov 2018-Jan 2021,False,False,Calibri,9.96,9605778,s1,main
